# Sentiment Analysis

In [4]:
from transformers import pipeline

In [ ]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

Device set to use cpu


In [3]:
import pandas as pd
from pathlib import Path

file_path = Path("data/processed")
politician = []
type_of_speech = []
file_name= []
text = []

for file in file_path.rglob("*.txt"):
    separated_file_name = str(file).split("/")

    politician.append(separated_file_name[2])
    type_of_speech.append(separated_file_name[3])
    file_name.append(separated_file_name[4])
    with open(file, 'r', encoding='utf-8') as f:
        content = f.read()
        text.append(content)

df = pd.DataFrame({
    'politician': politician,
    'type_of_speech': type_of_speech,
    'file_name': file_name,
    'text': text
})
df

,politician,type_of_speech,file_name,text
0,bernie_sanders,partisan_rally_speeches,rally_fighting_oligarchy_idaho_processed.txt,"bernie sanders ( 00:07 ): â¦ alexandria, they..."
1,bernie_sanders,partisan_rally_speeches,rally_2020_campaign_brooklyn_processed.txt,"sen. bernie sanders, i-vt., waves to supporter..."
2,bernie_sanders,partisan_rally_speeches,rally_fighting_oligarchy_greeley_processed.txt,bernie sanders ( 00:03 ): my clients right her...
3,bernie_sanders,partisan_rally_speeches,rally_dnc_2024_processed.txt,bernie sanders ( 00:01 ): thank you. thank you...
4,bernie_sanders,senate_floor_speeches,floor_2020_stimulus_2000_checks_processed.txt,... by brendan cole senior news reporter newsw...
5,bernie_sanders,senate_floor_speeches,floor_2025_trump_budget_processed.txt,prepared remarks: sanders speech on trump’s ba...
6,bernie_sanders,senate_floor_speeches,floor_2021_israel_gaza_ceasefire_processed.txt,bernie sanders: ( 00:02 ) reserving the right ...
7,bernie_sanders,senate_floor_speeches,floor_2025_what_oligarchs_want_processed.txt,prepared remarks: sanders on the senate floor:...
8,bernie_sanders,senate_floor_speeches,floor_2022_ukraine_crisis_processed.txt,prepared remarks: sanders senate floor speech ...
9,bernie_sanders,senate_floor_speeches,floor_2021_minimum_wage_processed.txt,"speaker 2: ( 01:14 ) the senator from vermont,..."


In [ ]:
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)